<a href="https://colab.research.google.com/github/selenerc/Business-metrics-and-KPIs/blob/master/Business_metrics_and_KPIs_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install PosgreSQL
%%capture
!wget -qO- https://www.postgresql.org/media/keys/ACCC4CF8.asc | apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ bionic-pgdg main" >/etc/apt/sources.list.d/pgdg.list
!apt -qq update
!apt -yq install postgresql-12 postgresql-client-12
!service postgresql start
# make calling psql shorter
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"  
!psql postgres -c "CREATE DATABASE root"  # now just !psql -c "..."
# load SQL extensions
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

In [4]:
# Download your data to local environment
!wget -q https://github.com/datacamp/data-analysis-in-sql-live-training/raw/master/data/user_data.csv
!wget -q https://github.com/datacamp/data-analysis-in-sql-live-training/raw/master/data/user_metadata.csv

In [5]:
# Create table
%%sql

DROP TABLE IF EXISTS user_sessions;
CREATE TABLE user_sessions(
 session_date date,
 user_id int,
 time_spent_in_mins int
);

COPY user_sessions
FROM '/content/user_data.csv' DELIMITER ',' CSV HEADER;

DROP TABLE IF EXISTS user_data;
CREATE TABLE user_data(
 user_id int,
 country char(3),
 age int
);

COPY user_data
FROM '/content/user_metadata.csv' DELIMITER ',' CSV HEADER;

 * postgresql+psycopg2://@/postgres


""


Exploring the tables.

In [6]:
%%sql

SELECT * 
FROM user_sessions 
LIMIT 5;

 * postgresql+psycopg2://@/postgres


,session_date,user_id,time_spent_in_mins
0,2020-01-01,1,127
1,2020-01-01,2,147
2,2020-01-01,5,106
3,2020-01-01,6,179
4,2020-01-01,7,143


In [7]:
%%sql

SELECT * 
FROM user_data 
LIMIT 5;

 * postgresql+psycopg2://@/postgres


,user_id,country,age
0,0,GER,23
1,1,IND,23
2,2,IND,43
3,3,GER,22
4,4,JPN,47


In [14]:
%%sql

-- Join the two tables together
SELECT *
FROM user_sessions s
INNER JOIN user_data u
  ON s.user_id = u.user_id
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,session_date,user_id,time_spent_in_mins,user_id,country,age
0,2020-01-01,1,127,1,IND,23
1,2020-01-01,2,147,2,IND,43
2,2020-01-01,5,106,5,GER,52
3,2020-01-01,6,179,6,GER,46
4,2020-01-01,7,143,7,GBR,59
5,2020-01-01,8,179,8,JPN,22
6,2020-01-01,10,94,10,GER,29
7,2020-01-01,11,76,11,RUS,45
8,2020-01-01,12,166,12,USA,49
9,2020-01-01,14,135,14,CHN,33


## Data overview

In [19]:
%%sql

-- Average age per country

SELECT country
    , ROUND(AVG(age),2) AS avg_age
FROM user_data u
GROUP BY country
ORDER BY avg_age;

 * postgresql+psycopg2://@/postgres


,country,avg_age
0,CHN,40.25
1,RUS,40.85
2,USA,41.30
3,FRA,41.38
4,GER,41.49
5,JPN,41.78
6,GBR,42.12
7,IND,42.27


In [21]:
%%sql

-- User count by country 

SELECT country
    , COUNT(DISTINCT user_id) AS user_count
FROM user_data u
GROUP BY country
ORDER BY country;

 * postgresql+psycopg2://@/postgres


,country,user_count
0,CHN,245
1,FRA,227
2,GBR,266
3,GER,247
4,IND,289
5,JPN,276
6,RUS,248
7,USA,263


In [29]:
%%sql

-- Daily active users (DAU)

SELECT session_date 
    , COUNT(DISTINCT user_id)
FROM user_sessions
GROUP BY session_date
ORDER BY session_date
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,session_date,count
0,2020-01-01,777
1,2020-01-02,755
2,2020-01-03,786
3,2020-01-04,760
4,2020-01-05,749
5,2020-01-06,751
6,2020-01-07,753
7,2020-01-08,778
8,2020-01-09,766
9,2020-01-10,766


In [27]:
%%sql
-- Monthly active users (MAU)

SELECT DATE_TRUNC('MONTH',session_date)::DATE AS Date
    , COUNT(DISTINCT user_id)
FROM user_sessions
GROUP BY Date
ORDER BY Date
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,date,count
0,2020-01-01,1472
1,2020-02-01,1621
2,2020-03-01,1723
3,2020-04-01,1879
4,2020-05-01,1995


In [46]:
%%sql

-- Stickiness = DAU / MAU    measures how often users engage on average.

WITH daus AS (
    SELECT session_date 
      , COUNT(DISTINCT user_id) AS dau
    FROM user_sessions
    GROUP BY session_date
),

maus AS (
    SELECT DATE_TRUNC('MONTH',session_date)::DATE AS Date
        , COUNT(DISTINCT user_id) AS mau
    FROM user_sessions
    GROUP BY Date
)

SELECT 
    ROUND(AVG(dau::NUMERIC/mau),2) AS stickiness
FROM daus
JOIN maus
  ON DATE_TRUNC('MONTH', daus.session_date ) = maus.Date
;

 * postgresql+psycopg2://@/postgres


,stickiness
0,0.52


## New users per day

In [8]:
%%sql

WITH regist_date AS (
    SELECT 
        user_id
        , MIN(session_date) AS reg_date
    FROM user_sessions
    GROUP BY user_id
)

SELECT 
    DATE_TRUNC('MONTH', reg_date)::DATE AS month
    , COUNT(user_id) AS monthly_new_users
FROM regist_date
GROUP BY month
ORDER BY month;

 * postgresql+psycopg2://@/postgres


,month,monthly_new_users
0,2020-01-01,1472
1,2020-02-01,166
2,2020-03-01,147
3,2020-04-01,152
4,2020-05-01,124


## GROWTH

In [27]:
%%sql

WITH regist_date AS (
    SELECT 
        user_id
        , MIN(session_date) AS reg_date
    FROM user_sessions
    GROUP BY user_id
),

new_users AS (
    SELECT 
        DATE_TRUNC('MONTH', reg_date)::DATE AS month
        , COUNT(user_id) AS monthly_new_users
    FROM regist_date
    GROUP BY month
)

SELECT 
    month
    , monthly_new_users
    , LAG(monthly_new_users) OVER (ORDER BY month) AS previous
    , ROUND((monthly_new_users - LAG(monthly_new_users) OVER (ORDER BY month))::NUMERIC / 
    LAG(monthly_new_users) OVER (ORDER BY month),2) AS growth_rate
FROM new_users;

 * postgresql+psycopg2://@/postgres


,month,monthly_new_users,previous,growth_rate
0,2020-01-01,1472,NaN,None
1,2020-02-01,166,1472.0,-0.89
2,2020-03-01,147,166.0,-0.11
3,2020-04-01,152,147.0,0.03
4,2020-05-01,124,152.0,-0.18


## Retained users

In [51]:
%%sql

-- months in which each user is active
WITH months_active AS (
    SELECT DISTINCT DATE_TRUNC('MONTH', session_date)::DATE AS active_month
        , user_id
    FROM user_sessions
),

retained AS (
    SELECT 
        previous.active_month
        , previous.user_id
        , current.user_id IS NOT NULL AS retained
    FROM months_active previous
    LEFT JOIN months_active current
      ON previous.user_id = current.user_id
      AND previous.active_month = (current.active_month + INTERVAL '1 MONTH')
)

SELECT 
    active_month AS month
    , COUNT(user_id) AS total_users
    , SUM(CASE WHEN retained THEN 1 ELSE 0 END) AS retained_users
    , ROUND(SUM(CASE 
          WHEN retained THEN 1 END)::NUMERIC / COUNT(*),3)
          AS ratention_rate
FROM retained
GROUP BY active_month;

 * postgresql+psycopg2://@/postgres


,month,total_users,retained_users,ratention_rate
0,2020-01-01,1472,0,None
1,2020-02-01,1621,1455,0.898
2,2020-03-01,1723,1559,0.905
3,2020-04-01,1879,1667,0.887
4,2020-05-01,1995,1816,0.910


## Average age of churners in Germany and Japan

In [71]:
%%sql 
WITH months_active AS (
    SELECT DISTINCT DATE_TRUNC('MONTH', session_date)::DATE AS active_month
        , user_id
    FROM user_sessions
),

retained AS (
    SELECT 
        previous.active_month
        , previous.user_id
        , current.user_id IS NULL AS churned
    FROM months_active previous
    LEFT JOIN months_active current
      ON previous.user_id = current.user_id
      AND previous.active_month = (current.active_month + INTERVAL '1 MONTH')
)

SELECT 
    country 
    , churned
    , ROUND(AVG(age),1) AS avg_age
FROM retained
JOIN user_data 
  ON retained.user_id = user_data.user_id
WHERE country IN ('GER', 'JPN')
GROUP BY country, churned;

 * postgresql+psycopg2://@/postgres


,country,churned,avg_age
0,JPN,False,41.7
1,JPN,True,41.9
2,GER,True,41.6
3,GER,False,41.4
